## ARP Spoofing detector  
### with Decision Tree

In [1]:
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as mt

In [2]:
#탐지할 데이터 셋 설정
raw_data = pd.read_csv('mitm-arpspoofing-1-dec.csv',low_memory=False)
raw_data.head()

,frame.number,ip.src,ip.dst,eth.src,eth.dst,arp.src.hw_mac,arp.dst.hw_mac,icmp,ip.proto,tcp.srcport,tcp.dstport,tcp.len,tcp.seq,tcp.ack,tcp.flags.push,tcp.flags.reset,tcp.flags.syn,tcp.flags.fin,tcp.stream,label
0,1,192.168.0.14,224.0.0.251,a8:66:7f:0e:3f:54,48:4b:aa:2c:d8:f9,NaN,NaN,NaN,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,NaN,NaN,a8:66:7f:0e:3f:54,48:4b:aa:2c:d8:f9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,3,192.168.0.13,192.168.0.1,bc:1c:81:4b:ae:ba,88:36:6c:d7:1c:56,NaN,NaN,icmp,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,4,192.168.0.1,192.168.0.13,88:36:6c:d7:1c:56,bc:1c:81:4b:ae:ba,NaN,NaN,icmp,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,5,192.168.0.14,224.0.0.251,a8:66:7f:0e:3f:54,a8:2b:b9:d6:5d:9c,NaN,NaN,NaN,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [3]:
#데이터 전처리
ip_src = []
ip_dst = []
eth_src = []
eth_dst = []
arp_src = []
arp_dst = []
ip_proto = []
        
for each in raw_data['ip.src']:
    if each == '192.168.0.16':
        ip_src.append(1)
    elif each == '192.168.0.13':
        ip_src.append(2)
    else:
        ip_src.append(0)
        
for each in raw_data['ip.dst']:
    if each == '192.168.0.16':
        ip_dst.append(1)
    elif each == '192.168.0.13':
        ip_dst.append(2)
    else:
        ip_dst.append(0)
        
for each in raw_data['eth.src']:
    if each == 'f0:18:98:5e:ff:9f':
        eth_src.append(1)
    else:
        eth_src.append(0)
        
for each in raw_data['eth.dst']: 
    if each == 'f0:18:98:5e:ff:9f':
        eth_dst.append(1)
    else:
        eth_dst.append(0)
        
for each in raw_data['arp.src.hw_mac']:
    if each == 'f0:18:98:5e:ff:9f':
        arp_src.append(1)
    else:
        arp_src.append(0)
        
for each in raw_data['arp.dst.hw_mac']:
    if each == 'bc:1c:81:4b:ae:ba':
        arp_dst.append(1)
    elif each == '48:4b:aa:2c:d8:f9':
        arp_dst.append(2)
    else:
        arp_dst.append(0)
        
for each in raw_data['ip.proto']:
    if each == '1':
        ip_proto.append(1)
    elif each == '6':
        ip_proto.append(6)
    else:
        ip_proto.append(0)


raw_data['ip.src'] = ip_src
raw_data['ip.dst'] = ip_dst
raw_data['eth.src'] = eth_src
raw_data['eth.dst'] = eth_dst
raw_data['arp.src.hw_mac'] = arp_src
raw_data['arp.dst.hw_mac'] = arp_dst
raw_data['ip.proto'] = ip_proto



train_pre = raw_data[['ip.src','ip.dst','eth.src','eth.dst','arp.src.hw_mac','arp.dst.hw_mac']]
train_pre

,ip.src,ip.dst,eth.src,eth.dst,arp.src.hw_mac,arp.dst.hw_mac
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,2,0,0,0,0,0
3,0,2,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
65763,1,2,0,0,0,0
65764,1,2,0,0,0,0
65765,1,2,0,0,0,0
65766,2,1,0,0,0,0


In [4]:
#훈련셋, 테스트셋 분리
x_train, x_test, y_train, y_test = train_test_split(train_pre,raw_data[['label']],random_state=0)

In [5]:
#DT 알고리즘 적용
dt_clf = DecisionTreeClassifier(criterion='entropy',max_depth = 4, random_state=0)
dt_clf = dt_clf.fit(x_train,y_train)
y_pred = dt_clf.predict(x_test)

In [6]:
#교차검증
from sklearn.model_selection import cross_val_score, cross_validate

#각 폴드의 스코어 
scores = cross_val_score(dt_clf, x_train, y_train, cv = 5)
scores

array([0.99989864, 1.        , 1.        , 1.        , 0.99989863])

In [7]:
#교차 검증 스코어 및 평균
pd.DataFrame(cross_validate(dt_clf, x_train, y_train, cv =5))
print('cross validation avg: ', scores.mean())

cross validation avg:  0.9999594546651434


In [8]:
#정확도 출력
print('Train_Accuracy:{}'.format(dt_clf.score(x_train,y_train)))

Train_Accuracy:0.9999797267161334


In [9]:
accuracy = mt.accuracy_score(y_test, y_pred)
recall = mt.recall_score(y_test, y_pred)
precision = mt.precision_score(y_test, y_pred)
f1_score = mt.f1_score(y_test, y_pred)

print('Accuracy: ', format(accuracy))
print('Recall: ', format(recall))
print('Precision: ', format(precision))
print('F1_score: ', format(f1_score))

Accuracy:  1.0
Recall:  1.0
Precision:  1.0
F1_score:  1.0


In [10]:
#탐지 결과 출력
print("<Result>")
for each in y_pred :
    print(each)

<Result>
[1 0 1 ... 1 0 0]
1
0
1
0
0
0
1
1
1
0
1
0
1
0
1
0
1
1
0
0
0
1
1
0
0
1
1
0
1
0
1
1
0
1
1
0
1
0
1
0
0
1
1
0
1
0
0
1
1
1
0
1
1
1
0
0
1
0
0
1
0
1
0
1
0
0
0
1
1
0
1
0
1
0
0
0
0
0
1
0
0
1
0
1
0
1
0
1
1
0
1
0
1
1
0
0
0
1
1
0
0
1
1
0
0
0
0
1
0
1
1
1
0
0
1
0
1
0
1
1
0
1
0
0
1
1
0
1
0
0
1
0
0
1
0
1
0
0
0
1
0
1
1
0
1
1
1
0
0
0
1
0
1
1
0
1
0
0
1
1
1
1
1
0
0
0
0
0
0
1
0
0
1
1
0
1
0
0
0
1
0
0
0
0
1
1
1
0
0
0
0
0
0
0
1
0
1
1
0
0
0
1
1
1
1
1
0
0
1
0
0
1
0
0
0
1
1
0
1
1
1
1
0
0
1
0
0
0
1
0
0
0
0
1
0
0
1
1
1
0
0
0
0
0
0
1
1
1
0
0
0
0
0
1
1
1
0
0
0
1
1
0
1
1
0
1
1
1
0
0
1
0
0
1
1
0
0
1
1
0
1
0
0
0
0
0
0
0
0
1
1
0
0
1
0
1
0
0
1
0
1
1
1
1
0
1
0
1
1
0
0
0
0
0
0
0
1
1
1
1
0
1
0
0
0
1
0
0
1
1
1
1
0
0
1
1
0
0
0
1
1
0
0
0
0
1
1
1
0
1
1
0
0
1
1
1
0
0
1
0
0
1
1
1
0
0
0
1
1
0
0
0
0
1
1
1
0
0
0
1
1
1
0
1
1
1
0
0
1
0
0
0
1
0
0
1
0
0
0
0
0
0
1
0
1
1
0
1
1
0
0
0
0
0
0
1
0
0
0
1
1
0
1
0
1
1
0
1
1
1
0
0
1
0
0
1
0
0
0
1
0
0
0
1
0
1
0
1
1
1
1
0
1
0
1
0
0
1
0
1
0
1
1
0
1
0
0
0
1
1
0
1
1
1
1
1
0
1
0
1
0
0
0
1
0
0
1